# Behavioral Cloning Code Notebook

First , I need to import all model I needed.

In [1]:
import keras
import pandas as pd
from keras.preprocessing import image
from keras.models import Model
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers import BatchNormalization, Input,Cropping2D,Lambda,Conv2D
from keras.optimizers import Adam
from keras import losses
import numpy as np
from sklearn.utils import shuffle
from PIL import Image
import random
import cv2
from keras.layers import Lambda


Using TensorFlow backend.


Read the data from our csv log file and save the information in a new file

In [2]:
data = pd.read_csv('data/data/driving_log.csv')
data['center']  = data['center'].apply(lambda x: 'data/data/' + x )
data['left']  = data['left'].apply(lambda x: 'data/data/'+ x.strip() )
data['right']  = data['right'].apply(lambda x: 'data/data/'+ x.strip() )

columns = ['center', 'left' , 'right', 'steering']
new_data = data[columns]
new_data.to_csv('new_data.csv')

Load the new file. I will get 10% of the data as valid data. The rest data will be used for training

In [2]:
new_data = pd.read_csv('new_data.csv')
new_data = shuffle(new_data)
n = len(new_data)
train_data = new_data[:int(0.9*n)]
valid_data = new_data[int(0.9*n):]

x_valid = valid_data['center'].values
y_valid = valid_data['steering'].values




I will use images from all cameras. For image from left cameras, I will add the steering angel by 0.25. For the right cameras, I will do the oppsite

In [3]:
m = len(train_data)
adjust_angel = 0.25
image_size = [160,320,3]

x_train  = np.zeros((3*m),dtype = 'O')
y_train  = np.zeros((3*m))

x_train[ : m] = train_data.center.values 
y_train[ : m] = train_data.steering.values

x_train[ m: 2*m] = train_data.left.values 
y_train[ m: 2*m] = train_data.steering.values + adjust_angel

x_train[ 2*m: 3*m] = train_data.right.values 
y_train[ 2*m: 3*m] = train_data.steering.values - adjust_angel

x_train, y_train = shuffle(x_train,y_train)

There are the fuctions I used later.

In [5]:
#To resize and normalize the image.
def preprocess_image(img):
    from keras.backend import tf as ktf
    img = ktf.image.resize_images(img, (64, 64) )
    img = 2* ((img / 255) - 0.5)
    return img


In [6]:
#Generator for validation
def get_valid_batch(x, y, batch_size = 32):
    num_batch = len(x)// batch_size
    x_batch = np.zeros((batch_size, image_size[0],image_size[1],image_size[2]))
    x = x[:num_batch * batch_size]
    y = y[:num_batch * batch_size]
    while 1:       
        for i in range(num_batch):
            files = x[i * batch_size :(i+1) * batch_size]
            y_batch = y[i * batch_size :(i+1) * batch_size]   
            for j in range(batch_size):
                #img = np.array(Image.open(files[j]))
                img = cv2.imread(files[j])
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)                
                x_batch[j] = img                    
            yield  x_batch, y_batch

In order to get more training data, my generator for training is  a bit complex. 
I used a lot image augumentation. 
I just random pick the images from my training set. 
Then I will randomly flip , change brightness, and shift the image. For the shifted image, I will compensate the angel.
And because most of time for our driving, angel is zero. So i just drop some of the data if it's angel is less than 0.1. 
This drop probability will be reduced as the training epoch rise.

In [7]:
#Generator for training.
shred = 100
def get_train_batch(x, y, batch_size = 32):
    x , y = shuffle(x, y)
    x_batch = np.zeros((batch_size, image_size[0],image_size[1],image_size[2]))
    y_batch = np.zeros((batch_size))
    while 1:       
        for i in range(batch_size):
            keep_prob = 0
            while keep_prob == 0:
                #random pick 1 data from my whole data set
                index = random.randint(0,len(x)) -1
                files, angel = x[index], y[index]
                img = cv2.imread(files)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      
                #data augumentaion            
                img, angel = random_flip(img, angel) #random flip            
                

                if abs(angel) < 0.1:
                    jj = random.randint(0,100)
                    if jj >shred:
                        keep_prob = 1
                else:
                    keep_prob = 1
            img, angel = shift_image(img, angel) #random shift
            img = augment_brightness(img) #random change brightness        
            x_batch[i] ,y_batch[i]= img, angel
            
        yield  shuffle(x_batch, y_batch)

In [8]:
def random_flip(img, angel):
    jj = random.randint(0,100)
    if jj >50:
        img = np.fliplr(img)
        angel = -1 * angel
    return img, angel

def augment_brightness(img):
    img = np.array(img, dtype = np.float32)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    #img = np.array(img, dtype = np.float32)
    random_bright = .5+np.random.uniform()
    img[: , : , 2] = img[: , : , 2] * random_bright
    img[: , : , 2] [img[: , : , 2]>255] = 255
    img = np.array(img,dtype = np.uint8)
    img = cv2.cvtColor(img,cv2.COLOR_HSV2RGB)
    return img

def shift_image(image,angel):
    shift_x = 100*np.random.uniform() - 50 # x axis use range of +-50 
    shift_y = 30*np.random.uniform() - 15 #y axis use range of +-15
    angel = angel + shift_x*0.005 # compensate the angel
    trans_m = np.float32([[1,0,shift_x],[0,1,shift_y]])
    image = cv2.warpAffine(image , trans_m , (320 , 160))
    return image, angel

For my architecture, I'm using 5 layers of convolution and 4 layers of dense.  I used dropout layer between dense layers.

In [9]:
pic_inputs = Input(shape = (160,320,3,), dtype = 'int32', name = 'pic_input')
pic_cropped = Cropping2D(cropping = ((50,30), (0,0)),input_shape = (160,320,3), name = 'pic_cropped')(pic_inputs)
pic_resize = Lambda(preprocess_image, name = 'pic_resized')(pic_cropped) #64*64

conv1 = Conv2D(filters = 48,kernel_size = (5,5) ,strides = (2,2), padding = 'same', activation = 'relu', name = 'conv1')(pic_resize)  #32*32
conv2 = Conv2D(filters = 36,kernel_size = (5,5),strides=(2,2), padding = 'same', activation = 'elu',name = 'conv2')(conv1) #16*16
conv3 = Conv2D(filters = 48,kernel_size = (5,5),strides=(2,2),  padding = 'same',activation = 'elu',name = 'conv3')(conv2) #8*8
conv4 = Conv2D(filters = 64,kernel_size = (3,3),strides=(2,2),  padding = 'same',activation = 'elu',name = 'conv4')(conv3) #4*4
#conv5 = Conv2D(filters = 80,kernel_size = (3,3),strides=(2,2),  padding = 'same',activation = 'elu',name = 'conv5')(conv4) #2*2


flat = Flatten()(conv4)

dense1 = Dense(100, activation = 'relu', name = 'dense1')(flat)
drop_out1 = Dropout(0.2)(dense1)

dense2 = Dense(50, activation = 'relu', name = 'dense2')(drop_out1)
drop_out2 = Dropout(0.2)(dense2)

dense3 = Dense(20, activation = 'relu', name = 'dense3')(drop_out2)
drop_out3 = Dropout(0.2)(dense3)

steering_angel = Dense(1, activation = None, name = 'output')(drop_out3)

model = Model(inputs = pic_inputs , outputs = steering_angel)
opt = keras.optimizers.Adam(lr = 0.0001)
model.compile(optimizer = opt, loss = 'mean_squared_error')

My computer went dead when I use Model.fit_generator..I don't know why..So I wrote this function to call modle.train_on_batch to avoid that.

In [10]:
def train_one_epoch():
    loss_total = 0
    data = get_train_batch(x_train, y_train, batch_size = batch_size)
    for i in range(num_steps_train):
        x_tr, y_tr = next(data)
        model.train_on_batch(x_tr, y_tr)
        if i%100 == 0 :
            loss = model.evaluate(x= x_tr,y = y_tr,verbose = 0)
            print ('Batch({}/{}: loss{})'.format(i,num_steps_train,loss))

The loss didn't go down much after 2 epochs of training. So I'm not going to train more and see how this goes.
And result is very good. My model is working perfect on track2.

In [13]:
batch_size = 128
opt = keras.optimizers.Adam(lr = 0.0001)
nb_epoch = 8
num_steps_train = (len(x_train) // batch_size) *8
num_steps_valid = (len(x_valid) // batch_size) 

for e in range(nb_epoch):
    print("epoch {}".format(e+1))
    train_one_epoch()
    train_data = get_valid_batch(x_train, y_train, batch_size = batch_size)
    train_loss = model.evaluate_generator(train_data, steps = num_steps_train/8)
    shred = 100 / (e+1)
    print ('Train loss:{}'.format(train_loss))
    valid_data = get_valid_batch(x_valid, y_valid, batch_size = batch_size)
    valid_loss = model.evaluate_generator(valid_data, steps = num_steps_valid)
    print ('Valid loss:{}'.format(valid_loss))

epoch 1
Batch(0/1352: loss0.028633666690438986)
Batch(100/1352: loss0.021348596084862947)
Batch(200/1352: loss0.020161375403404236)
Batch(300/1352: loss0.031038138549774885)
Batch(400/1352: loss0.021555493585765362)
Batch(500/1352: loss0.018212802708148956)
Batch(600/1352: loss0.01502295769751072)
Batch(700/1352: loss0.03213645378127694)
Batch(800/1352: loss0.02592579834163189)
Batch(900/1352: loss0.025934227742254734)
Batch(1000/1352: loss0.02331148274242878)
Batch(1100/1352: loss0.027378769824281335)
Batch(1200/1352: loss0.0241343155503273)
Batch(1300/1352: loss0.027837557718157768)
Train loss:0.027955196213704594
Valid loss:0.02482956958313783
epoch 2
Batch(0/1352: loss0.02223508246243)
Batch(100/1352: loss0.019850264070555568)
Batch(200/1352: loss0.022905481979250908)
Batch(300/1352: loss0.023467320716008544)
Batch(400/1352: loss0.02203707443550229)
Batch(500/1352: loss0.022404426708817482)
Batch(600/1352: loss0.026634466368705034)
Batch(700/1352: loss0.023705987259745598)
Batch(80

KeyboardInterrupt: 

In [14]:
model.save('model2.h5')

This is the code which will cause my p2 instance to die after running..

In [ ]:
train_data = get_train_batch(x_train, y_train, batch_size = batch_size)
valid_data = get_valid_batch(x_valid, y_valid, batch_size = batch_size)
model.fit_generator(generator = train_data, steps_per_epoch= num_steps_train, epochs =1 , validation_data = valid_data, validation_steps = num_steps_valid)